In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers

In [2]:
# Read data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
for_result_test_data = pd.read_csv("test.csv")

In [3]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [43]:
# Remove un-use columns

def drop_columns(df, list_columns=[]):
  return df.drop(list_columns, axis =1 )

list_columns =  ["PassengerId", "Name", "Ticket", "Cabin"]
train_data = drop_columns(train_data, list_columns)
list_columns_test =  ["PassengerId", "Name", "Ticket", "Cabin"]
test_data = drop_columns(test_data, list_columns_test)


In [44]:
# Remove nan in exist columns

train_data.fillna(train_data.mean(),inplace=True)
test_data.fillna(test_data.mean(),inplace=True)

In [45]:
# Split purpose

train_data_y = train_data.Survived
list_columns_survived =  ["Survived"]
train_data = drop_columns(train_data, list_columns_survived)

In [46]:
# one-hot vector

def one_hot_vector(df, list_OHV_columns):
  for column_name in list_OHV_columns:
    df = pd.get_dummies(df, columns=[column_name], prefix="oh"+column_name, sparse=True)
  return df

one_hot_columns = ["Pclass", "Embarked"]
train_data = one_hot_vector(train_data, one_hot_columns)
test_data = one_hot_vector(test_data, one_hot_columns)


In [47]:
# label encoding
sex_le = LabelEncoder()
train_data.Sex = sex_le.fit_transform(train_data.Sex)
test_data.Sex = sex_le.fit_transform(test_data.Sex)

In [48]:
# Standardization

sc = StandardScaler()
train_data.Age = sc.fit_transform(train_data.Age.values.reshape(-1, 1))
test_data.Age = sc.transform(test_data.Age.values.reshape(-1,1))

In [49]:
# to numpy
X = train_data.values
y = train_data_y.values

In [50]:
# Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [51]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [52]:
model = Sequential()
model.add(Dense(100, activation = "relu", input_dim = X_train.shape[1], name = "layer_1", kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(50, activation = "sigmoid", name = "layer_2", kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(25, activation = "sigmoid", name = "layer_3" , kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1, activation = "sigmoid", name ="output_layer")) 
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc'])


In [53]:
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
23/23 [==============================] - 2s 3ms/step - loss: 1.0893 - acc: 0.3890
Epoch 2/100
23/23 [==============================] - 0s 3ms/step - loss: 0.8080 - acc: 0.5014
Epoch 3/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7739 - acc: 0.6039
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7602 - acc: 0.6053
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7630 - acc: 0.6025
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7359 - acc: 0.6067
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7256 - acc: 0.6110
Epoch 8/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7216 - acc: 0.6194
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 0.7234 - acc: 0.6110
Epoch 10/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7144 - acc: 0.6039
Epoch 11/100
23/23 [==============================] - 0s 4m

In [54]:

score = model.evaluate(X_train, y_train)
print("")
print("Train loss:{0}".format(score[0]))
print("Train accuracy:{0}".format(score[1]))
t_score = model.evaluate(X_test, y_test)
print("")
print("Test loss:{0}".format(t_score[0]))
print("Test accuracy:{0}".format(t_score[1]))



23/23 [==============================] - 0s 2ms/step - loss: 0.4953 - acc: 0.8048

Train loss:0.4953378736972809
Train accuracy:0.8047752976417542
6/6 [==============================] - 0s 2ms/step - loss: 0.4676 - acc: 0.8101

Test loss:0.46760988235473633
Test accuracy:0.8100558519363403


In [55]:
result = model.predict(test_data.values)

14/14 [==============================] - 0s 2ms/step


In [56]:
df_suv = pd.DataFrame(result, columns = ["Survived"])

In [57]:
df_suv.Survived = round(df_suv.Survived)
df_suv.Survived = df_suv.Survived.astype('int8')

In [58]:
df_r = pd.concat([for_result_test_data, df_suv], axis=1)

In [59]:
df_r.drop(columns=[ "Name", "Ticket", "Fare", "Cabin", "Embarked",'Sex', 'Age', 'SibSp', 'Parch', 'Pclass'],inplace =True)

In [60]:
df_r

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [61]:
df_r.to_csv("test_result-2.csv", index = False)

In [62]:
df_r.columns

Index(['PassengerId', 'Survived'], dtype='object')

In [63]:
df_r

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [64]:
df_suv

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,1
415,0
416,0


In [65]:
test_data

,Sex,Age,SibSp,Parch,Fare,ohPclass_1,ohPclass_2,ohPclass_3,ohEmbarked_C,ohEmbarked_Q,ohEmbarked_S
0,1,0.369449,0,0,7.8292,0,0,1,0,1,0
1,0,1.331378,1,0,7.0000,0,0,1,0,0,1
2,1,2.485693,0,0,9.6875,0,1,0,0,1,0
3,1,-0.207709,0,0,8.6625,0,0,1,0,0,1
4,0,-0.592481,1,1,12.2875,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1,0.044131,0,0,8.0500,0,0,1,0,0,1
414,0,0.715743,0,0,108.9000,1,0,0,1,0,0
415,1,0.677266,0,0,7.2500,0,0,1,0,0,1
416,1,0.044131,0,0,8.0500,0,0,1,0,0,1
